# INQUISIT PYTHON SCRIPT NOTEBOOK

In [1]:
# IMPORTS
import pandas as pd
import glob
import os
from shutil import copyfile

## Add JPG To Prescene Excel Files

In [ ]:
# Prescene image names in the Excel files are missing the file extension.

files = glob.glob('participant_files/*.xlsx')

for fp in files:
    if 'PreScene' not in fp:
        copyfile(fp, 'output_cleaned/%s' % fp.split(os.sep)[1])
        continue

    df = pd.read_excel(fp)
    df['PreScene'] = df['PreScene'].astype(str)
    df['PreScene'] += '.jpg'
    df.to_excel('output_cleaned/%s' % fp.split(os.sep)[1], index=False)

## Excel To Inquisit Code

In [ ]:
# Convert the excel files into inquisit code.
# if the column that is being converted contains an image name. Add the path to the image folder to its name.

files = glob.glob('output_cleaned/*.xlsx')

for fp in files:
    df = pd.read_excel(fp)
    print fp
    items = ""
    file_name = fp.split(os.sep)[1].split('.')[0]
    for col in df.columns.values:
        items += '<item %s_%s>\n' % (file_name, col)
        i = 1
        for index, row in df[col].iteritems():
            if '.jpg' in str(row) or '.jpeg' in str(row):
                items += '/%d = \"images/%s\"\n' % (i, row)
            else:
                items += '/%d = \"%s\"\n' % (i, row)
            i += 1

        items += '</item>\n'
        fp = open('output_inquisit/%s.txt' % file_name, 'w')
        fp.write(items)
        fp.close()

## Pre-scene Inquisit

In [ ]:
# Create a unqiue pre-scene file for each ordering that calls the unique subject file ordering

a = "<include>\n" \
    "/ file = \"../../prescene_{0}.iqx\"\n" \
    "/ file = \"../../SubjectFiles/{1}.txt\"\n" \
    "</include>\n\n" \
    "< picture background_img_prescene_{0}>\n" \
    "/ items = item.{1}_PreScene\n" \
    "/ select = sequence\n" \
    "/ erase = false\n" \
    "/ size = (50%%, 50%%)\n" \
    "/ position = (50%%, 35%%)\n" \
    "</picture>\n\n"

b = "<text prescene_sub_{1}>\n" \
    "/ items = item.{1}_Sub\n" \
    "/ select = sequence\n" \
    "/ color = white\n" \
    "/ size = (0, 0)\n" \
    "/ erase = false\n" \
    "</text>"

files = glob.glob('output_cleaned/*PreScene.xlsx')

for name in files:
    file_name = name.split(os.sep)[1].split('.')[0]

    participant_and_block = file_name[:-9]

    block = file_name.split('_')[1].lower()

    inquisit_file = open('inquisit/prescene/%s.txt' % participant_and_block, 'w')
    inquisit_file.write(a.format(block, file_name))
    inquisit_file.write(b.format(block, file_name))
    inquisit_file.close()

## Exp Inquisit

In [ ]:
# Create the block orderings for each participant. Must be unique per subject ordering since it must call a specific file based
# on the subject number.

var = "<include>\n" \
      "/ file = \"prescene/Sub1_B1_Prac.txt\"\n" \
      "/ file = \"../survey.iqx\"\n" \
      "/ file = \"prescene/Sub{0}_B1.txt\"\n" \
      "/ file = \"prescene/Sub{0}_B2.txt\"\n" \
      "/ file = \"prescene/Sub{0}_B3.txt\"\n" \
      "</include>\n\n" \
      "<expt>\n" \
      "/blocks=[1=instruction_block_prac_prescene_b1;2=b1_prac_prescene;3=survey;4=instruction_block_prescene_b1;"\
        "5=b1_prescene;6=instruction_block_prescene_b2;7=b2_prescene;8=instruction_block_prescene_b3;9=b3_prescene;"\
        "10=demographics_block;11=worker_id_block"\
      "</expt>"

num_participants = 16

for i in range(1, num_participants + 1):
    content = var.format(i)
    fp = open('inquisit/exp/Sub%d.txt' % i, 'w')
    fp.write(content)
    fp.close()

## Include Inquisit

In [ ]:
# Include file required since Main Inquisit file <batch> tag does not support paths.
# One required by ordering, calls the Exp file for that ordering.

var = "<include>\n" \
      "/ file = \"exp/Sub%d.txt\"\n" \
      "</include>"

num_participants = 16
for i in range(1, num_participants + 1):
    fp = open('inquisit/include/Sub%d_include.iqx' % i, 'w')
    fp.write(var % i)
    fp.close()

## Main Inquisit

In [ ]:
# Call the correct Include file based on the participant file.
# 16 possible orderings. First 16 participants use the first 16 orderings, then the counter will go back to 1 and 17th participant
# will use ordering #1, 18th will use ordering #2 and so on.

num_of_subjects = 1000

var = "<batch>\n" \
      "/ subjects = (%i of " \
      + str(num_of_subjects) + \
      ")\n" \
      "/ file = \"Sub%i_include.iqx\"\n" \
      "</batch>\n\n"

fp = open('rachel_inquisit.iqx', 'w')

j = 1
for i in range(1, num_of_subjects + 1):
    fp.write(var % (i, j))
    j = j + 1 if j < 16 else 1

fp.close()